### Feature extraction from a real file

In [4]:
import numpy as np 
import pandas as pd

In [5]:
df = pd.read_csv('../TextFiles/smsspamcollection.tsv', sep='\t')

In [6]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [7]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [8]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df['message']

In [11]:
y = df['label']

In [12]:
# creating train and test suit
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
# Transform documents into feature vectors
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
count_vect = CountVectorizer() # this module allows us to do all the transformations

In [15]:
X # we sill pass this to the count vectorizer and transform it.

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

### Two steps to transform with the Count Vectorizer

1-FIT VECTORIZER TO THE DATA (build a vocab, count the number of words...)

2-TRANSFORM THE ORIGINAL TEXT MESSAGE --> VECTOR

In [17]:
# OPTION 1: Two steps (fit, transform)
# count_vect.fit(X_train) # build vocab, count nuber of words
# X_train_counts = count_vect.transform(X_train) # transform the text

# OPTION 2: All in one step (fit and transform)
X_train_counts = count_vect.fit_transform(X_train) # build vocab, count words and transforms

In [18]:
# this is a huge sparse matrix
X_train_counts # 3733 messages x 7082 unique words in those messages

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [19]:
X_train.shape

(3733,)

In [21]:
# we have the messages and the vocab count (see last step in 01-Feature-Extraction-from-Text2.ipynb)
X_train_counts.shape 

(3733, 7082)